In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'test'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 13:43:28.469233+00:00,Joe,0,1,COMPLETE,2020-01-01 14:59:41.608354+00:00,2,0 days 01:16:13.139121,4573,49408,1,0,0,0,0,0,1,0
1,REPAIR,START,2020-01-01 14:59:41.608354+00:00,Jane,0,4,COMPLETE,2020-01-04 18:38:11.639016+00:00,5,3 days 03:38:30.030662,272310,53981,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 19:33:48.900814+00:00,1,1,10,COMPLETE,2020-01-01 20:34:39.406353+00:00,11,0 days 01:00:50.505539,3650,70428,1,0,0,1,0,0,0,0
4,REPAIR,START,2020-01-01 20:34:39.406353+00:00,1,1,13,COMPLETE,2020-01-02 09:05:17.659036+00:00,14,0 days 12:30:38.252683,45038,74079,1,0,1,2,0,0,0,0
6,DIAGNOSIS,START,2020-01-02 07:47:21.536525+00:00,Clark,2,19,COMPLETE,2020-01-02 08:45:47.275476+00:00,20,0 days 00:58:25.738951,3505,28041,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,QUALITY_CONTROL,START,2022-06-30 11:11:14.473595+00:00,Jane,890,8017,COMPLETE,2022-06-30 13:45:04.355487+00:00,8018,0 days 02:33:49.881892,9229,40274,1,1,1,0,0,3,0,0
2669,QUALITY_CONTROL,START,2022-06-30 12:13:46.487067+00:00,Joe,889,8008,COMPLETE,2022-06-30 19:40:59.136223+00:00,8009,0 days 07:27:12.649156,26832,44026,1,1,1,1,0,1,1,0
2673,DIAGNOSIS,START,2022-06-30 22:25:05.097535+00:00,Karsten,891,8020,COMPLETE,2022-06-30 23:15:20.606227+00:00,8021,0 days 00:50:15.508692,3015,80705,1,0,0,0,0,0,0,1
2674,REPAIR,START,2022-06-30 23:15:20.606227+00:00,Jane,891,8023,COMPLETE,2022-07-01 07:28:48.982352+00:00,8024,0 days 08:13:28.376125,29608,83720,1,0,1,0,0,1,0,1


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:08<2:11:54,  8.88s/it]

  0%|▎                                                                                 | 4/892 [00:08<25:18,  1.71s/it]

  1%|▉                                                                                | 11/892 [00:18<20:45,  1.41s/it]

  1%|█▏                                                                               | 13/892 [00:18<16:10,  1.10s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████ | 882/892 [01:46<00:00, 10.10it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▎| 884/892 [01:46<00:00,  8.54it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 886/892 [01:46<00:00,  8.93it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:47<00:00,  7.26it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:47<00:00,  8.33it/s]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('-2.506504543275551150151882010')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(11715.73627851244)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:20<5:07:40, 20.72s/it]

  0%|▏                                                                               | 2/892 [00:21<2:17:21,  9.26s/it]

  0%|▎                                                                               | 3/892 [00:23<1:25:23,  5.76s/it]

  0%|▎                                                                               | 4/892 [00:27<1:16:29,  5.17s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 883/892 [01:45<00:00,  9.18it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 886/892 [01:45<00:00, 10.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:46<00:00, 10.34it/s]

100%|███████████████████████████████████████████████████████████████████████████████▊| 890/892 [01:46<00:00,  8.77it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:46<00:00,  8.38it/s]

In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('-0.5261534040482819460928109987')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(10069.33583957274)

In [11]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:24<6:04:22, 24.54s/it]

  0%|▏                                                                               | 2/892 [00:25<2:42:27, 10.95s/it]

  0%|▎                                                                               | 4/892 [00:26<1:04:10,  4.34s/it]

  1%|▍                                                                                 | 5/892 [00:26<45:15,  3.06s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 883/892 [01:44<00:01,  7.98it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:45<00:00, 11.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████▊| 890/892 [01:45<00:00,  9.52it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  9.39it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  8.45it/s]

In [12]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

Decimal('0.01531916808543196785938074119')

In [13]:
np.mean(get_pscores(likelihoods_R_S))

np.float64(12239.56349989538)

In [14]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:29<7:20:42, 29.68s/it]

  0%|▏                                                                               | 2/892 [00:31<3:17:48, 13.34s/it]

  1%|▊                                                                                 | 9/892 [00:32<29:17,  1.99s/it]

  1%|▉                                                                              | 11/892 [00:58<1:11:18,  4.86s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 883/892 [01:45<00:00,  9.01it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▎| 885/892 [01:45<00:00,  9.54it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▌| 887/892 [01:46<00:00,  6.79it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:46<00:00,  9.21it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:46<00:00,  8.36it/s]

In [15]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

Decimal('-0.1841028279985302640747921883')

In [16]:
np.mean(get_pscores(likelihoods_R))

np.float64(14037.585059739311)

In [17]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:20<5:05:31, 20.57s/it]

  0%|▏                                                                               | 2/892 [00:21<2:14:44,  9.08s/it]

  0%|▎                                                                               | 3/892 [00:23<1:25:40,  5.78s/it]

  0%|▎                                                                                 | 4/892 [00:24<56:29,  3.82s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 884/892 [01:44<00:00, 10.77it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 886/892 [01:44<00:00,  9.70it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:45<00:00,  8.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████▉| 891/892 [01:45<00:00,  8.91it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  8.47it/s]

In [18]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('-0.4843691824499295964620401170')

In [19]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(10774.643642903931)

In [20]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:23<5:48:34, 23.47s/it]

  0%|▏                                                                               | 2/892 [00:25<2:36:56, 10.58s/it]

  1%|▊                                                                                 | 9/892 [00:25<23:22,  1.59s/it]

  1%|▊                                                                                 | 9/892 [00:36<23:22,  1.59s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 883/892 [01:42<00:00,  9.58it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▌| 887/892 [01:42<00:00, 11.51it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 889/892 [01:42<00:00, 11.21it/s]

100%|███████████████████████████████████████████████████████████████████████████████▉| 891/892 [01:42<00:00,  9.63it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:43<00:00,  8.64it/s]

In [21]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('-0.2175879577168559507020794817')

In [22]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(10456.507443409986)

In [23]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:20<5:03:17, 20.42s/it]

  0%|▏                                                                               | 2/892 [00:23<2:34:42, 10.43s/it]

  1%|▉                                                                                | 10/892 [00:25<22:56,  1.56s/it]

  1%|▉                                                                                | 11/892 [00:41<50:20,  3.43s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 884/892 [01:45<00:01,  7.54it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▎| 885/892 [01:45<00:00,  7.26it/s]

100%|███████████████████████████████████████████████████████████████████████████████▊| 890/892 [01:45<00:00, 12.31it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00, 10.61it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  8.44it/s]

In [24]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('0.2849778793072964446352518627')

In [25]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(8775.551640050819)

In [26]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:23<5:49:38, 23.54s/it]

  0%|▏                                                                               | 2/892 [00:29<3:18:40, 13.39s/it]

  1%|▉                                                                                | 11/892 [00:46<47:02,  3.20s/it]

  1%|█                                                                                | 12/892 [00:47<42:23,  2.89s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 885/892 [01:44<00:00,  9.66it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▌| 887/892 [01:44<00:00, 10.41it/s]

100%|███████████████████████████████████████████████████████████████████████████████▊| 890/892 [01:44<00:00, 11.57it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:44<00:00, 11.06it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:44<00:00,  8.50it/s]

In [27]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('0.3122759763320390200742041941')

In [28]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(8253.235312848425)

In [29]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)